Création du dataframe d'entrainement final à partir des données
- Tweets format score
- Données BTC (API Coingecko) labelisées via Triple Barrier Method
- Données macroéconomique (API FRED)

In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from get_data_API import get_economic_data
from get_sentiment_score import tweets_to_sentiment_scores

## Données macroéconomique

In [ ]:
# clef API FRED obtenue via https://fred.stlouisfed.org/docs/api/api_key.html
FRED_API_KEY = os.getenv('FRED_API_KEY')

series_list = ['DFF', # FF Federal Funds
            'NFINCP', # NFCP Nonfinancial commercial paper outstanding
            'FINCP', # FCP Financial commercial paper outstanding
                'DPRIME', # PRIME Bank prime loan
                'DPCREDIT', # DWPC Discount window primary credit
                'DTWEXBGS', # Nominal Broad U.S. Dollar Index 
                'CPIAUCSL', #Consumer price index
                'DGS3MO', # Market Yield on U.S. Treasury Securities 3 mois
                'DGS1', # Market Yield on U.S. Treasury Securities 1 an
                'DGS30' # Market Yield on U.S. Treasury Securities 30 ans
                ]

economic_df = get_economic_data(series_id_list=series_list, api_key=FRED_API_KEY)

## Données X scraped

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

tweets_df = pd.read_csv('data/tweets/tweets_data_CoinDesk.csv')
tweets_text = tweets_df['tweet_text'].tolist()

# prediction des scores
scores = tweets_to_sentiment_scores(tweets_text)

# ajout scores au df
tweets_df['negative'] = scores[:, 0].tolist()
tweets_df['neutral'] = scores[:, 1].tolist()
tweets_df['positive'] = scores[:, 2].tolist()

print(tweets_df)


## Données Bitcoin + Labels TBM via Genetic optimization